<a href="https://colab.research.google.com/github/Usually-zz/2021_IMC/blob/main/gmm_em.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class GMM(object):
  
  def __init__(self, X, k=3):
    self.X = np.asarray(X)
    
    # dimension
    self.n, self.p = X.shape
    #print(n)
    #print(p)
    # number of mixtures
    self.k = k

  def _init(self):
    # init mixture means/sigmas
    self.nj = torch.zeros(self.k)
    self.wj = torch.ones(self.k) / self.k
    self.muj = torch.ones(self.k, self.p)
    self.coj = torch.zeros(self.k, self.p, self.p)
    self.rij = torch.ones(self.k, self.n)
    self.mvnj = torch.zeros(self.k, self.n) 
    self.pred = torch.zeros(self.n)
    #muj, coj, rij, mvnj

  def train(self, itermax=100):
    
    self._init() 
    self.gmmKmeansInitial()
    
    llv = [0.0]
    for iter in range(itermax):
      lli = self.estep()
      self.mstep()
      
      print('Iteration', iter + 1, 'Likelihood: ', lli)
        
      if abs(llv[-1] - lli) < 1e-4:
        break 
      
      llv.append(lli)
    return llv[1:]

  def estep(self):
    #mvnj = torch.zeros(k, X.shape[0]) 
    for j in range(self.k):
      self.mvnj[j] = (self.wj[j] * stats.multivariate_normal(self.muj[j], self.coj[j]).pdf(self.X))
      
    bot = torch.sum(self.mvnj, 0)
    self.rij = self.mvnj / bot
    logl = torch.sum(torch.log(bot))
      
    self.pred = torch.max(self.rij, 0).indices
    
    return logl

  def mstep(self):

    d = torch.from_numpy(np.array(self.X))
    
    self.nj = torch.sum(self.rij, 1)
    self.wj = self.nj / sum(self.nj) ## same as nj / n
  
    for j in range(self.k):
      self.muj[j] = torch.sum(d.T * self.rij[j], 1) / self.nj[j]

    for j in range(self.k):
      ker = (d - self.muj[j]).T * self.rij[j]
      self.coj[j] = torch.matmul(ker, ker.T)/self.nj[j]

  def gmmKmeansInitial(self):
    
    kmeans = KMeans(n_clusters=self.k, random_state=0).fit(self.X)
    self.muj = torch.tensor(kmeans.cluster_centers_)
  
    for j in range(self.k):
      indx = kmeans.labels_ == j
      d = self.X[indx]
      self.nj[j] = sum(indx)
      #muj.append(np.array(d.mean(axis=0)))
      self.coj[j] = torch.from_numpy(np.cov(d.T))
    
    self.wj = self.nj / torch.sum(self.nj)

In [ ]:
import torch
import numpy as np
import scipy.stats as stats ## for mvn
from sklearn.cluster import KMeans
import seaborn as sns

In [ ]:
## try iris data
k = 3
iris = sns.load_dataset("iris")
X = iris.iloc[:,:4]
y = iris.iloc[:,4]

gmm = GMM(X)
lls = gmm.train()

Iteration 1 Likelihood:  tensor(-197.3202)
Iteration 2 Likelihood:  tensor(-193.0827)
Iteration 3 Likelihood:  tensor(-191.4538)
Iteration 4 Likelihood:  tensor(-189.7725)
Iteration 5 Likelihood:  tensor(-188.4172)
Iteration 6 Likelihood:  tensor(-186.6134)
Iteration 7 Likelihood:  tensor(-184.5180)
Iteration 8 Likelihood:  tensor(-183.0643)
Iteration 9 Likelihood:  tensor(-182.3152)
Iteration 10 Likelihood:  tensor(-181.4987)
Iteration 11 Likelihood:  tensor(-180.6420)
Iteration 12 Likelihood:  tensor(-180.4911)
Iteration 13 Likelihood:  tensor(-180.4788)
Iteration 14 Likelihood:  tensor(-180.4806)
Iteration 15 Likelihood:  tensor(-180.4832)
Iteration 16 Likelihood:  tensor(-180.4847)
Iteration 17 Likelihood:  tensor(-180.4854)
Iteration 18 Likelihood:  tensor(-180.4857)
Iteration 19 Likelihood:  tensor(-180.4858)
Iteration 20 Likelihood:  tensor(-180.4859)


In [ ]:
gmm.pred

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 2,
        0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])

In [ ]:
gmm.rij.shape

torch.Size([3, 150])

In [ ]:
from sklearn.metrics import confusion_matrix
targets = torch.cat((torch.tensor([1]).repeat(50), torch.tensor([2]).repeat(50), torch.tensor([0]).repeat(50)))
confusion_matrix(targets, gmm.pred)

array([[50,  0,  0],
       [ 0, 50,  0],
       [ 5,  0, 45]])

In [ ]:
import sklearn.datasets

## generate data
n = 1000; p = 15; k = 5 #np.random.randint(2, 7)
#XX = torch.tensor(sklearn.datasets.make_spd_matrix(n, p))

In [ ]:
## generate random data
ww = torch.rand(k); wj = ww/sum(ww)
muj = torch.zeros((k, p))
coj = torch.zeros((k, p, p))

In [ ]:
for j in range(k):
  ## means
  muj[j] = 10 * torch.randn(p)

  ## covariance matrix
  mat = torch.rand(p, p)
  mat = torch.mm(mat, mat.t())
  coj[j].add_(torch.eye(p))

In [ ]:
samples = np.zeros((n, p+1))
u = np.random.uniform(size=n)

In [ ]:
for i in range(n):
  for j in range(k):
    if u[i] < sum(wj[:(j+1)]):
      samples[i] = np.append(np.random.multivariate_normal(muj[j], coj[j], 1), [j])
      break

In [ ]:
tgmm = GMM(samples[:,:p], k)

1000
15


In [ ]:
tlls = tgmm.train()

Iteration 1 Likelihood:  tensor(-27722.5117)
Iteration 2 Likelihood:  tensor(-27722.3984)
Iteration 3 Likelihood:  tensor(-27722.3984)


In [ ]:
tlls

[tensor(-27722.5117), tensor(-27722.3984)]

In [ ]:
np.unique(tgmm.pred)

array([0, 1, 2, 3, 4])

In [21]:
class GMM(object):
    def __init__(self, X, k=2):
        # dimension
        X = np.asarray(X)
        self.m, self.n = X.shape
        self.data = X.copy()
        # number of mixtures
        self.k = k
        
    def _init(self):
        # init mixture means/sigmas
        self.mean_arr = np.asmatrix(np.random.random((self.k, self.n)))
        self.sigma_arr = np.array([np.asmatrix(np.identity(self.n)) for i in range(self.k)])
        self.phi = np.ones(self.k)/self.k
        self.w = np.asmatrix(np.empty((self.m, self.k), dtype=float))
        #print(self.mean_arr)
        #print(self.sigma_arr)
    
    def fit(self, tol=1e-4):
        self._init()
        num_iters = 0
        ll = 1
        previous_ll = 0
        while(ll-previous_ll > tol):
            previous_ll = self.loglikelihood()
            self._fit()
            num_iters += 1
            ll = self.loglikelihood()
            print('Iteration %d: log-likelihood is %.6f'%(num_iters, ll))
        print('Terminate at %d-th iteration:log-likelihood is %.6f'%(num_iters, ll))
    
    def loglikelihood(self):
        ll = 0
        for i in range(self.m):
            tmp = 0
            for j in range(self.k):
                #print(self.sigma_arr[j])
                tmp += sp.stats.multivariate_normal.pdf(self.data[i, :], 
                                                        self.mean_arr[j, :].A1, 
                                                        self.sigma_arr[j, :]) *\
                       self.phi[j]
            ll += np.log(tmp) 
        return ll
    
    def _fit(self):
        self.e_step()
        self.m_step()
        
    def e_step(self):
        # calculate w_j^{(i)}
        for i in range(self.m):
            den = 0
            for j in range(self.k):
                num = sp.stats.multivariate_normal.pdf(self.data[i, :], 
                                                       self.mean_arr[j].A1, 
                                                       self.sigma_arr[j]) *\
                      self.phi[j]
                den += num
                self.w[i, j] = num
            self.w[i, :] /= den
            assert self.w[i, :].sum() - 1 < 1e-4
            
    def m_step(self):
        for j in range(self.k):
            const = self.w[:, j].sum()
            self.phi[j] = 1/self.m * const
            _mu_j = np.zeros(self.n)
            _sigma_j = np.zeros((self.n, self.n))
            for i in range(self.m):
                _mu_j += (self.data[i, :] * self.w[i, j])
                _sigma_j += self.w[i, j] * ((self.data[i, :] - self.mean_arr[j, :]).T * (self.data[i, :] - self.mean_arr[j, :]))
                #print((self.data[i, :] - self.mean_arr[j, :]).T * (self.data[i, :] - self.mean_arr[j, :]))
                #print(_sigma_j)
                print((self.data[i, :] - self.mean_arr[j, :]).T)
                print((self.data[i, :] - self.mean_arr[j, :]))
                adfd
                #print(_sigma_j.shape)
            self.mean_arr[j] = _mu_j / const
            self.sigma_arr[j] = _sigma_j / const
        #print(self.sigma_arr)

In [3]:
import numpy as np
import scipy as sp

X = np.random.multivariate_normal([0, 3], [[0.5, 0], [0, 0.8]], 20)
X = np.vstack((X, np.random.multivariate_normal([20, 10], np.identity(2), 50)))
X.shape

(70, 2)

In [4]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline



In [22]:
gmm = GMM(X)
gmm.fit()

[[-0.1361679 ]
 [ 2.91257148]]
[[-0.1361679   2.91257148]]


NameError: ignored